<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/MLP_HPT(cifar10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q wandb

     |████████████████████████████████| 1.8MB 15.7MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 143kB 41.7MB/s 
     |████████████████████████████████| 143kB 52.0MB/s 
     |████████████████████████████████| 174kB 51.2MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import argparse
from tensorflow import keras

from wandb.keras import WandbCallback
import wandb

wandb.login()

True

In [16]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

In [17]:
print("x shape")
print(f"train: {x_train.shape}, val: {x_val.shape}, test: {x_test.shape}")
print("y shape")
print(f"train: {y_train.shape}, val: {y_val.shape}, test: {y_test.shape}")

x shape
train: (40000, 32, 32, 3), val: (10000, 32, 32, 3), test: (10000, 32, 32, 3)
y shape
train: (40000, 1), val: (10000, 1), test: (10000, 1)


In [18]:
class MLP(tf.keras.Model):
    def __init__(self, out_dim, hid_dim, n_layer, act):
        super(MLP, self).__init__()
        self.out_dim = out_dim
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.act = act

        self.flatten = tf.keras.layers.Flatten(input_shape=(32, 32))
        self.fc = tf.keras.layers.Dense(self.out_dim)
        self.linears = [
            tf.keras.layers.Dense(self.hid_dim) for i in range(self.n_layer - 1)]

        self.fc2 = tf.keras.layers.Dense(self.out_dim)

        if self.act == 'relu':
            self.act = tf.keras.layers.Activation('relu')

    def call(self, x):
        x = self.flatten(x)
        x = self.act(self.fc(x))
        for fc in self.linears:
            x = self.act(fc(x))
        x = self.fc2(x)
        return x

net = MLP(10, 100, 4, 'relu')

In [23]:
def experiment():
    config_defaults = {
        'n_layer': 4,
        'out_dim': 10,
        'hid_dim': 50,
        'act': 'relu',
        'lr': 0.001,
        'mm': 0.9,
        'epochs': 2,
        'batch_size': 128
    }

    wandb.init(project='DL', config=config_defaults, magic=True)
    args = wandb.config
    
    # Prepare trainloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare testloader 
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    keras.backend.clear_session()
    
    net = MLP(args.out_dim, args.hid_dim, args.n_layer, args.act)

    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(learning_rate=args.lr, momentum=args.mm)
    metrics = tf.keras.metrics.SparseCategoricalAccuracy()

    net.compile(optimizer, loss, metrics)

    result = net.fit(
        trainloader, 
        epochs=2, 
        validation_data=valloader, 
        batch_size=128,
        callbacks=[WandbCallback(save_model=False)])

    test_result = net.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])

In [24]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'n_layer': {
            'values': [4, 5, 6]
        },
        'out_dim': {
            'values': [10]
        },
        'hid_dim': {
            'values': [50, 100, 150]
        },
        'act': {
            'values': ['relu']
        },
        'lr': {
            'values': [0.001]
        },
        'mm': {
            'values': [0.9]
        },
        'epochs': {
            'values': [2]
        },
        'batch_size': {
            'values': [128]
        }
    }
}

import wandb
sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=experiment)

Create sweep with ID: 5panrz0h
Sweep URL: https://wandb.ai/jeongmin/uncategorized/sweeps/5panrz0h


wandb: Agent Starting Run: 4kqvdl9n with config:
wandb: 	act: relu
wandb: 	batch_size: 128
wandb: 	epochs: 2
wandb: 	hid_dim: 50
wandb: 	lr: 0.001
wandb: 	mm: 0.9
wandb: 	n_layer: 4
wandb: 	out_dim: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/2
313/313 [==============================] - 4s 11ms/step - loss: 2.3378 - sparse_categorical_accuracy: 0.1006 - val_loss: 2.3021 - val_sparse_categorical_accuracy: 0.1013
Epoch 2/2
79/79 [==============================] - 0s 4ms/step - loss: 2.2740 - sparse_categorical_accuracy: 0.1000


epoch,1
loss,2.29106
sparse_categorical_accuracy,0.09967
val_loss,2.27487
val_sparse_categorical_accuracy,0.1013
_runtime,12
_timestamp,1626695895
_step,1
best_val_loss,2.27487
best_epoch,1


epoch,▁█
loss,█▁
sparse_categorical_accuracy,█▁
val_loss,█▁
val_sparse_categorical_accuracy,▁▁
_runtime,▁█
_timestamp,▁█
_step,▁█


...

wandb: Agent Starting Run: sjwxg6eg with config:
wandb: 	act: relu
wandb: 	batch_size: 128
wandb: 	epochs: 2
wandb: 	hid_dim: 150
wandb: 	lr: 0.001
wandb: 	mm: 0.9
wandb: 	n_layer: 6
wandb: 	out_dim: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/2
313/313 [==============================] - 3s 9ms/step - loss: 10.8672 - sparse_categorical_accuracy: 0.1022 - val_loss: 10.8171 - val_sparse_categorical_accuracy: 0.0918
Epoch 2/2
79/79 [==============================] - 0s 4ms/step - loss: 10.9556 - sparse_categorical_accuracy: 0.1000


epoch,1
loss,10.89347
sparse_categorical_accuracy,0.10205
val_loss,10.9039
val_sparse_categorical_accuracy,0.0918
_runtime,11
_timestamp,1626696078
_step,1
best_val_loss,10.8171
best_epoch,0


epoch,▁█
loss,▁█
sparse_categorical_accuracy,█▁
val_loss,▁█
val_sparse_categorical_accuracy,▁▁
_runtime,▁█
_timestamp,▁█
_step,▁█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
